In [1]:
import os
import pandas as pd
from tqdm import tqdm


# 遍历当前目录文件夹，存为list
def get_file_list(dir):
    file_list = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

In [2]:
# 清洗数据，并按尺码分类
def df_clean(file):
    df = pd.read_excel(file, sheet_name="Sheet1")
    df_new = df[
        [
            "订单号",
            "订单状态",
            "发货时间",
            "SKC",
            "属性集",
            "发货单号",
            "正品数量",
            "次品数量",
            "退货时间",
        ]
    ].copy()
    # 带copy()取消警告
    df_new["发货单号"] = df_new["发货单号"].astype(str)

    def classify_size(size):
        if any(x in size for x in ["0XL", "1XL", "2XL", "3XL", "4XL"]):
            return "0XL-4XL"
        else:
            return "S-XL"

    df_new.loc[:, "尺码分类"] = df_new.loc[:, "属性集"].apply(classify_size)
    df_new.sort_values(by=["发货时间", "订单号", "尺码分类"], inplace=True)
    return df_new


# 将每个订单号的正品数量求和，其余保留第一个属性
def group_sum(df: pd.DataFrame):
    # 获取所有列名，排除用于分组的列和需要求和的列
    group_cols = ['订单号']
    sum_cols = ['正品数量']
    first_cols = [col for col in df.columns if col not in group_cols + sum_cols]

    # 创建聚合字典
    agg_dict = {col: 'first' for col in first_cols}
    agg_dict.update({col: 'sum' for col in sum_cols})

    # 按订单号分组，并应用聚合函数
    result = (
        df.groupby(group_cols)
        .agg(agg_dict)
        .reset_index()
        .sort_values(by=['发货时间', '订单号', '尺码分类'])
    )
    return result

In [3]:
# 示例数据
data = {
    '订单号': [1, 1, 2, 2, 3],
    '属性': ['属性A', '属性B', '属性A', '属性B', '属性A'],
    '数量': [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data)

# 按订单号分组，并对数量求和，保留第一个属性
result = df.groupby('订单号').agg({'属性': 'first', '数量': 'sum'}).reset_index()

print(result)

   订单号   属性  数量
0    1  属性A  30
1    2  属性A  70
2    3  属性A  50


In [4]:
import pandas as pd

# 示例数据
data = {
    '订单号': [1, 2, 3],
    '客户名': ['Alice', 'Bob', 'Charlie'],
    '产品': ['Product1', 'Product2', 'Product3'],
    '属性集': ['黑色-S', '黑色-0XL', '黑色-M'],
}

df = pd.DataFrame(data)


# 定义分类函数
def classify_size(size):
    if any(x in size for x in ['0XL', '1XL', '2XL', '3XL', '4XL']):
        return '0XL-4XL'
    else:
        return 'S-XL'


# 使用 .loc 方法新增列
df.loc[:, '尺码分类'] = df['属性集'].apply(classify_size)
print(df)

   订单号      客户名        产品     属性集     尺码分类
0    1    Alice  Product1    黑色-S     S-XL
1    2      Bob  Product2  黑色-0XL  0XL-4XL
2    3  Charlie  Product3    黑色-M     S-XL


In [5]:
# 拼接一个月三十个表
from datetime import datetime

df_list = []
file_list = get_file_list('./11月')
for file in tqdm(file_list):

    df_new = df_clean(file)

    df_new = group_sum(df_new)
    df_list.append(df_new)
df_summary = pd.concat(df_list)
df_summary.sort_values(by=['发货时间', '订单号', '尺码分类'], inplace=True)
sheet_names = {'S-XL': '常规牛奶丝露胶S-XL', '0XL-4XL': '胖码牛奶丝露脐0XL-4XL'}
with pd.ExcelWriter(
    f'./结果/30表_{datetime.now().strftime("%m")}月.xlsx', engine='openpyxl'
) as writer:

    for key, value in sheet_names.items():

        df_summary[df_summary['尺码分类'] == key].to_excel(
            writer, index=False, sheet_name=value
        )

100%|██████████| 30/30 [00:05<00:00,  5.97it/s]


In [6]:
import os
import pandas as pd
from tqdm import tqdm

file_path = f'./结果/30表_{datetime.now().strftime("%m")}月.xlsx'
file_list = get_file_list("./两表")
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a") as writer:
    for file in tqdm(file_list):
        df_new = df_clean(file)
        sheet_name = os.path.basename(file).replace(".xlsx", "")
        df_summary = group_sum(df_new)
        if not df_summary.empty:
            df_summary.to_excel(writer, sheet_name=sheet_name, index=False)

print("DataFrames 已添加到 '30表.xlsx'")

100%|██████████| 2/2 [00:00<00:00,  6.80it/s]


DataFrames 已添加到 '30表.xlsx'


In [1]:
import pandas as pd

# 示例数据
data = {
    "订单号": [1, 2, 3, 4, 5],
    "属性列": ["A", "B", "A", "B", "A"],
    "数量": [10, 20, 30, 40, 50],
}

df_summary = pd.DataFrame(data)

# 根据属性列的值将 DataFrame 分成两个 DataFrame
df_A = df_summary[df_summary["属性列"] == "A"]
df_B = df_summary[df_summary["属性列"] == "B"]

print("DataFrame A:")
print(df_A)
print("\nDataFrame B:")
print(df_B)

DataFrame A:
   订单号 属性列  数量
0    1   A  10
2    3   A  30
4    5   A  50

DataFrame B:
   订单号 属性列  数量
1    2   B  20
3    4   B  40
